In [144]:
import csv
import json
from lxml import etree
import spacy
from spacy.gold import GoldParse
from spacy.scorer import Scorer

# récupérer les annotations manuelles sur un fichier au format "Inline XML"
def recup_annotations_manuelles(fichier):
    annotations = []
    parser = etree.XMLParser(recover=True)
    root = etree.parse(fichier, parser)
    balise_EN = "de.tudarmstadt.ukp.dkpro.core.api.ner.type.NamedEntity"
    
    for el in root.iter(balise_EN):
        annotations.append((int(el.attrib["begin"]), int(el.attrib["end"]), el.attrib["value"]))
    return annotations

# récupérer les annotations et le texte à partir d'un fichier au format json particulier (exporter format UIMA CAS JSON sur INCEPTION)
def recup_annotations_texte(fichier):
    dico = lire_json(fichier_texte_ann_manuelles)
    annots = dico["_views"]["_InitialView"]["NamedEntity"]
    annotations = []
    for ann in annots:
        annotations.append((ann["begin"], ann["end"], ann["value"]))
    texte = dico["_referenced_fss"]["1"]["sofaString"]
    return annotations, texte

# récupérer le texte d'un fichier ".txt"
def recup_texte(fichier):
    with open(fichier, "r", encoding="utf-8") as f:
        texte = f.read()
    return texte

# récupérer les annotations d'un fichier csv avec les entêtes "Indice_Début", "Indice Fin", "Tag"
def recup_annotations_csv(fichier):
    annotations = []
    with open(fichier, "r", encoding="utf-8", newline="") as csvfile:
        reader = csv.DictReader(csvfile, delimiter=",")
        for row in reader:
            #print(row['Tag'], row['Indice_Début'], row['Indice_Fin'])
            annotations.append((int(row['Indice_Début']), int(row['Indice_Fin']), row['Tag']))

    return annotations

# lecture d'un fichier json à partir de son chemin
# renvoie le dictionnaire correspondant
def lire_json(fichier):
    with open(fichier, "r", encoding="utf-8") as fin:
        dic = json.load(fin)
    return dic

# fonction d'évaluation pour Spacy
# prend en argument le model utilisé et les donnees à tester, sous la forme d'une liste de tuples (texte, annotations attendues)
def evaluate(ner_model, donnees):
    #scorer = Scorer()
    for input_, annot in donnees:
        scorer = Scorer()
        doc_gold_text = ner_model.make_doc(input_)
        # Gold : valeurs de référence
        gold = GoldParse(doc_gold_text, entities=annot)
        # Valeurs prédites par le modèle
        pred_value = ner_model(input_)
        #print(pred_value)
        for ent in pred_value.ents:
            print(ent.text, ent.start_char, ent.end_char, ent.label_)
        scorer.score(pred_value, gold)
    return scorer.scores

In [129]:
chemin_fichier = "Evaluation/Fichiers/5408048"
fichier_texte_ann_manuelles = "%s_texte_manuelles.json" % chemin_fichier

fichier_text_corr = "%s_correction.txt" % chemin_fichier

fichier_an_auto = "%s_annotations.csv" % chemin_fichier
fichier_an_auto_corr = "%s_annotations_correction.csv" % chemin_fichier


manuelles, texte = recup_annotations_texte(fichier_texte_ann_manuelles)

texte_corr = recup_texte(fichier_text_corr)
print(len(texte_corr))
auto = recup_annotations_csv(fichier_an_auto)
print(len(auto))
auto_corr = recup_annotations_csv(fichier_an_auto_corr)
print(len(auto_corr))

95450
224
227


In [145]:
versions = [
    (texte2, manuelles)
]

In [146]:
ner_model = spacy.load('fr_core_news_sm')
results = evaluate(ner_model, versions)
print(results["ents_per_type"])
print("-"*10)

Isis 112 116 PER
K  123 125 PER
Ministre du Commerce et’ de l’Industrie 350 389 ORG
Chambres de Commerce 425 445 MISC
Inventeurs de France 704 724 MISC
Paris 737 742 LOC
Syndicat des Inventeurs de France 948 981 ORG
Inventeurs de France 1436 1456 MISC
Loi de 1844 1586 1597 MISC
— 1600 1601 PER
Inventeurs de France 1964 1984 MISC
— 2036 2037 PER
Inventeurs 2627 2637 MISC
Loi 2889 2892 MISC
— 2903 2904 PER
Inventeurs 3152 3162 MISC
Art 3195 3198 MISC
— 3203 3204 PER
s’ 3273 3275 MISC
c’ 3515 3517 MISC
l’ 3624 3626 LOC
n’ 3739 3741 ORG
jusqu’ 4539 4545 MISC
Loi 4729 4732 MISC
— 4743 4744 PER
Inventeurs de France 5085 5105 MISC
Sociétés de Chemins de fer, de Navigation 5607 5648 ORG
canal de Suez 5653 5666 LOC
Crédit foncier 5671 5685 ORG
Inventeurs de France 6441 6461 MISC
Messieurs 6722 6731 PER
— 6914 6915 PER
Chambre s’ 7400 7410 LOC
M. le Ministre du Commerce d’ 7508 7537 PER
Messieurs 7683 7692 MISC
Inventeurs 9294 9304 MISC
Belgique 10048 10056 LOC
Messieurs 10705 10714 PER
Belgique

{}
----------


In [139]:
test = "Evaluation/Fichiers/5408048_correction.txt"

with open(test, "r", encoding="utf-8") as fin:
    texte2 = fin.read()